In [ ]:
import sys
import pandas as pd
import numpy as np

# Add the project root directory to Python path
project_root = r'c:\Users\pedro.hun\Documents\repos\option_monitor2'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.orchestrators.df_creator import DFCreator
from src.orchestrators.single_expiry_fitting import SingleExpiry

In [ ]:
df = DFCreator()

In [ ]:
final_df = df.apply_natural()

In [ ]:
final_df

In [ ]:
# final_df.dropna(inplace=True)
# for x in final_df['TTE_days'].unique():
#     fitting = SingleExpiry(days_to_expiry=x, df=final_df)
#     fitting.plot_svi()

In [ ]:
# orchestrator.py
from src.core.surfaces.ssvi2 import SSVICalibrator
from src.core.visualization.plottters import plot_ssvi_slices, plot_ssvi_heatmap

market_slices = final_df
calibrator = SSVICalibrator(config)
surface = calibrator.fit(market_slices)

plot_ssvi_slices(
    model=surface,
    expiries=[0.1, 0.25, 0.5, 1.0],
    market_points=surface.raw_slices()
)

plot_ssvi_heatmap(
    model=surface,
    t_min=min(surface.times),
    t_max=max(surface.times),
    k_min=-1.0,
    k_max=1.0
)